In [2]:
import matplotlib.animation as animation
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go

In [70]:
class Map:
    def __init__(self, src, x, y):
        raw_data = np.loadtxt(src, delimiter='\t')
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        # for i in range(len(self.vectors)):
        #     self.vectors[i]/=np.dot(self.vectors[i], self.vectors[i])**.5
    
    def getBaseVectors(self, count):
        # feature_matrix = self.vectors.copy()[:count]
        base_vectors = np.dot(self.vectors[:count], self.data_matrix)
        # z_scale = []
        # for i in range(len(z_matrix)):
        #     _size = np.dot(z_matrix[i], z_matrix[i])**.5
        #     z_matrix[i] /= _size
        #     feature_matrix = feature_matrix.T
        #     feature_matrix[i] *= _size
        #     feature_matrix = feature_matrix.T
        #     z_scale.append(_size)
        return base_vectors
    
    def reconstruct(self, base_vectors):
        return np.dot(self.vectors[:base_vectors.shape[0]].T, base_vectors)
    
    def plotComponent(self, id):
        img = self.vectors[id]
        img = np.reshape(img, self.shape) 
        fig = px.imshow(img, title = 'Komponent ' + str(id))
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self):
        fig = px.bar(self.weights, title = 'Wagi')
        return fig
    
    def plotSpectrum(self, id):
        fig = px.line(self.data, x = 'RamanShift', y = id)
        return fig
        
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt', 34, 34)

In [46]:
  
  # map_a.plotCovMatrix().show()
map_a.plotWeights().show()
# print(map_a.locToId(1,0))
# map_a.plotSpectrum(0).show()

In [97]:
z = map_a.getBaseVectors(1)
r = map_a.reconstruct(z)
print(map_a.vectors[:6].T[i])
px.line(z[0]).show()
px.line(np.dot(-1, z[0])).show()
i = 78
f = map_a.plotSpectrum(i)
f.add_scatter(x = map_a.raman_shift, y = r[i], mode='lines')
f.update_layout(hovermode="x")
f.show()

[-0.00085394  0.00442627 -0.01022937 -0.03857376  0.01464574 -0.00337301]


In [96]:
i = 5
z = map_a.getBaseVectors(10)
px.line(x = map_a.raman_shift, y = z[i]).show()
map_a.plotComponent(i).show()